# Orlando Criminal Activities

##### Presented by: Clark Jones, Umapathy Beema, Richard Morand <br> Dr. Spencer Lyon - Spring 2024 


Ojective: We will be exploring the criminal acts in Orlando, and create a model to showcase the types of activities done in the city. <br>

__Tools__ <br>
Collaboration: VSCode, Github, Discord <br>
Languages: Python, Julia
Packages/Libraries: Agents, OpenStreetMap, Pandas, etc.

In [ ]:
#import packages here
import Pkg
Pkg.add(["OpenStreetMapX","CSV", "DataFrames"])

In [26]:
using Agents, OpenStreetMapX
using  CSV, DataFrames

In [27]:
# load and analyze data
data = CSV.read("Data/filtered_data.csv", DataFrame)

Row,Case #,Incident Type,Number of Officers Involved,Suspect's Weapon,Georeferenced Column
,String15,String,Float64?,String31,String
1,22-174307,Vehicle Stop,2.0,Handgun,POINT (-81.430522027 28.559823981)
2,18-448868,Stolen Vehicle,1.0,Motor Vehicle,POINT (-81.444407991 28.426330019)
3,18-219116,Attempted Suicide,1.0,Handgun,POINT (-81.436934015 28.497201986)
4,20-397311,Domestic Disturbance,1.0,Handgun,POINT (-81.31615297 28.574676001)
5,10-503738,Armed Robbery,3.0,Handgun,POINT (-81.475974825 28.552001109)
6,13-022948,Drug Violation,2.0,Handgun,POINT (-81.398586976 28.552194982)
7,16-021812,Traffic Stop,1.0,Motor Vehicle,POINT (-81.454123826 28.596308986)
8,14-086291,Stalking,3.0,Motor Vehicle,POINT (-81.386192991 28.540632985)
9,15-291159,Suspicious Person,1.0,Hands/ASP,POINT (-81.335092645 28.522822186)


In [ ]:
# Function to convert Georeferenced Columm to Tuple 

function convert_to_tuple(str)
    test = str[7:end]
    test = strip(test, ['(',')'])
    test = split(test, ' ')

    return  (parse(Float64,test[1]), parse(Float64,test[2]))

end

convert_to_tuple (generic function with 1 method)

In [159]:
#Algorithms used
location = convert_to_tuple.(data[: , "Georeferenced Column"])

78-element Vector{Tuple{Float64, Float64}}:
 (-81.430522027, 28.559823981)
 (-81.444407991, 28.426330019)
 (-81.436934015, 28.497201986)
 (-81.31615297, 28.574676001)
 (-81.475974825, 28.552001109)
 (-81.398586976, 28.552194982)
 (-81.454123826, 28.596308986)
 (-81.386192991, 28.540632985)
 (-81.335092645, 28.522822186)
 (-81.444618041, 28.497712989)
 ⋮
 (-81.397620962, 28.544747996)
 (-81.307727984, 28.529062019)
 (-81.389041531, 28.538220076)
 (-81.429694969, 28.611062074)
 (-81.462643153, 28.490099922)
 (-81.269478007, 28.420598013)
 (-81.306244974, 28.518123007)
 (-81.388922977, 28.534726006)
 (-81.413406013, 28.601738987)

In [163]:
minmax(location[1], location[2])

((-81.444407991, 28.426330019), (-81.430522027, 28.559823981))

In [ ]:
# Begin ABM